In [17]:
import requests
from io import StringIO
import pandas as pd

def monthly_report_crawler(year, month):
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_{}_{}_0.html'.format(year, month)
    r = requests.get(url)
    r.encoding = 'big5'
    dfs = pd.read_html(StringIO(r.text), encoding='big5')
    count = 0
    for i in range(len(dfs)):
        if len(dfs[i]) > 1:
            if count == 0:
                df = dfs[i]
                cols = [i[1] for i in dfs[4].columns]
                count += 1
            else:
                df = pd.concat((df,dfs[i]), axis=0)
                count += 1
    df.columns = cols
    df = df[df['公司代號'] != '合計']
    return df


In [23]:
import os
import datetime
# build main dir
main_dir = os.path.join('data', 'monthly_report')
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)

for year in range(109,111):
    for month in range(1,13):
        try:
            df = monthly_report_crawler(year, month)
            df.to_csv(os.path.join(main_dir,'monthly_report_{}_{:02d}.csv'.format(year, month)), index=None, encoding='utf-8-sig')
            print('Download monthly_report_{}_{:02d} Successfully!'.format(year, month))
        except:
            print('Fail to Download monthly_report_{}_{:02d} !'.format(year, month))

Download monthly_report_109_01 Successfully!
Download monthly_report_109_02 Successfully!
Download monthly_report_109_03 Successfully!
Download monthly_report_109_04 Successfully!
Download monthly_report_109_05 Successfully!
Download monthly_report_109_06 Successfully!
Download monthly_report_109_07 Successfully!
Download monthly_report_109_08 Successfully!
Download monthly_report_109_09 Successfully!
Download monthly_report_109_10 Successfully!
Download monthly_report_109_11 Successfully!
Download monthly_report_109_12 Successfully!
Fail to Download monthly_report_110_01 !
Fail to Download monthly_report_110_02 !
Fail to Download monthly_report_110_03 !
Fail to Download monthly_report_110_04 !
Fail to Download monthly_report_110_05 !
Fail to Download monthly_report_110_06 !
Fail to Download monthly_report_110_07 !
Fail to Download monthly_report_110_08 !
Fail to Download monthly_report_110_09 !
Fail to Download monthly_report_110_10 !
Fail to Download monthly_report_110_11 !
Fail to D

In [2]:
def get_monthly_profit(ticker):
    main_dir = os.path.join('data', 'monthly_report')
    files = os.listdir(main_dir)
    result = pd.DataFrame()
    profit_list = []
    date_list = []
    for file in files:
        if '.csv' not in file:
            continue
        _, _, year, month = file.replace('.csv', '').split('_')
        df = pd.read_csv(os.path.join(main_dir, 'monthly_report_{}_{}.csv'.format(year, month)))
        profit = df[df['公司代號'] == ticker]['當月營收'].values[0]
        time = '{}-{}'.format(1911+int(year), month)
        profit_list.append(profit)
        date_list.append(time)
    result['date'] = date_list
    result['monthly profit'] = profit_list
    result['date'] = pd.to_datetime(result['date'])
    result = result.set_index('date')
    result = result.sort_index()
    return result

In [22]:
import os
import datetime
# build main dir
main_dir = os.path.join('data', 'monthly_report')
year, month = 109, 11
df = pd.read_csv(os.path.join(main_dir, 'monthly_report_{}_{}.csv'.format(year, month)))
df = df[df['公司代號'] == 2330]
table = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='#3474eb',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[df[col] for col in df.columns],
            fill_color='lavender',
            align='left'))
])
import plotly.offline as opy
table.update_layout(width=1200, height=300)
table.show()
plot_div = opy.plot(table, auto_open=False, output_type='div')